# Göttingen Campus Covid Publications 

This notebook creates, transforms and saves tables of COVID-19 publication data for researchers from the Göttingen Campus using Google BigQuery and the Dimensions COVID-19 data base.

Author: Andreas Lüschow

2021/07/08

-----

## Imports

In [1]:
# run imports
%run ../imports.ipynb

# import constants from constants notebook
%run ../constants.ipynb

# import methods from utils notebook
%run ../utils.ipynb

env: GOOGLE_APPLICATION_CREDENTIALS=../../../bigquery_credentials.json


## Setting up the Google BigQuery Client

In [2]:
client = get_bq_client()

-----

# Part 1: Create basic tables

## Create table with information about Göttingen Covid publications

Table: __GOE_COVID__

- SQL statement selects relevant fields from the Dimensions table of COVID-19 publications
- Both `authors` and `authors.grid_ids` fields need to be unnested in order to filter only those `grid_ids` belonging to institutions from the Göttingen Campus

In [3]:
sql = f"""
SELECT
    id,
    doi,
    title.preferred as title,
    year,
    journal.title as journal,
    publisher.name as publisher,
    resulting_publication_doi,
    pmid,
    pmcid,
    type,
    date,
    date_online,
    date_print,
    ANY_VALUE(open_access_categories_v2) AS open_access_categories_v2,
    ANY_VALUE(categories) AS categories,
    metrics.field_citation_ratio as field_citation_ratio,
    metrics.times_cited as times_cited,
    metrics.relative_citation_ratio as relative_citation_ratio,
    altmetrics.score as altmetrics_score,
    altmetrics.id as altmetrics_id,
FROM {DS_PUBLICATIONS} as t
LEFT JOIN 
    UNNEST (t.authors) as aut
LEFT JOIN
    UNNEST(aut.grid_ids) as aut_grid
WHERE
    aut_grid IN {GOE_CAMPUS_IDS}
GROUP BY
    id,
    doi,
    title,
    year,
    journal,
    publisher,
    resulting_publication_doi,
    pmid,
    pmcid,
    type,
    date,
    date_online,
    date_print,
    field_citation_ratio,
    times_cited,
    relative_citation_ratio,
    altmetrics_score,
    altmetrics_id
ORDER BY DATE
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_COVID)
)

In [4]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_COVID)}.csv")
df.head(5)

,id,doi,title,year,journal,publisher,resulting_publication_doi,pmid,pmcid,type,date,date_online,date_print,open_access_categories_v2,categories,field_citation_ratio,times_cited,relative_citation_ratio,altmetrics_score,altmetrics_id
0,pub.1126768313,10.2139/ssrn.3575633,Economic Preferences and Compliance in the Soc...,2020,SSRN Electronic Journal,Elsevier,None,None,None,preprint,2020,None,None,"[oa_all, green]","{'bra_v1': None, 'for_v1': {'first_level': {'c...",NaN,8,NaN,NaN,None
1,pub.1135164226,10.2139/ssrn.3738617,The Credit Composition of Global Liquidity,2020,SSRN Electronic Journal,Elsevier,None,None,None,preprint,2020,None,None,"[oa_all, green]","{'bra_v1': None, 'for_v1': {'first_level': {'c...",NaN,0,NaN,NaN,None
2,pub.1129925107,10.2139/ssrn.3661798,Mask Wars: China's Exports of Medical Goods in...,2020,SSRN Electronic Journal,Elsevier,None,None,None,preprint,2020,None,None,"[oa_all, green]","{'bra_v1': None, 'for_v1': {'first_level': {'c...",NaN,3,NaN,NaN,None
3,pub.1128004980,10.5414/alx02147e,Allergen immunotherapy in the current COVID-19...,2020,Allergologie select,Dustri-Verlgag Dr. Karl Feistle,None,32568272,None,article,2020-01-01,2020-05-28,2020-01-01,"[oa_all, green]","{'bra_v1': None, 'for_v1': None, 'hra_v1': Non...",NaN,8,NaN,3.0,96693033
4,pub.1130687096,10.5414/alx02166e,Use of biologicals in allergic and type-2 infl...,2020,Allergologie select,Dustri-Verlgag Dr. Karl Feistle,None,32915172,7480069,article,2020-01-01,2020-09-07,2020-01-01,"[oa_all, green]",{'bra_v1': {'values': ['Clinical Medicine and ...,NaN,12,NaN,NaN,None


## Create table with author information for each publication ID

Table: __GOE_AUTHORS__

- SQL statement selects author information from Dimensions table for each publication that is present in the __GOE_COVID__ table
- Note: This does also include authors not from the Göttingen Campus

In [5]:
sql = f"""
SELECT
  ds.id,
  authors
FROM
  {DS_PUBLICATIONS} AS ds
WHERE
  EXISTS (
  SELECT
    1
  FROM
    {GOE_COVID} AS goe
  WHERE
    (`ds`.`id` = `goe`.`id`)
  )
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_AUTHORS)
)

In [6]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_AUTHORS)}.csv")
df.head(5)

,id,authors
0,pub.1134912700,"[{'first_name': 'Johannes', 'last_name': 'Ring..."
1,pub.1131624687,"[{'first_name': 'Daniel', 'last_name': 'Kiefl'..."
2,pub.1133882352,"[{'first_name': 'C', 'last_name': 'Zampetidis'..."
3,pub.1131796221,"[{'first_name': 'Marco', 'last_name': 'Schiavo..."
4,pub.1137262430,"[{'first_name': 'Metodi V.', 'last_name': 'Sta..."


-----

# Part 2: Authors and Affiliations

## Create table with counts of author affiliations

Table: __GOE_AUTHORS_AFFILIATION__

- SQL statement selects all affiliations and the frequency of their occurrence for each `first_name`-`last_name` combination, ordered by frequency.

In [7]:
sql = f"""
SELECT
    a.first_name,
    a.last_name,
    grid_iden,
    count(grid_ids) as freq
FROM {GOE_AUTHORS} as goe
LEFT JOIN 
    UNNEST(goe.authors) AS a
LEFT JOIN 
    UNNEST(a.grid_ids) AS grid_iden
GROUP BY
    a.first_name,
    a.last_name,
    a.researcher_id,
    grid_iden
ORDER BY freq DESC
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_AUTHORS_AFFILIATION)
)

In [8]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_AUTHORS_AFFILIATION)}.csv")
df.head(5)

,first_name,last_name,grid_iden,freq
0,Stefan,Pöhlmann,grid.418215.b,44
1,Stefan,Pöhlmann,grid.7450.6,41
2,Markus,Hoffmann,grid.418215.b,40
3,Markus,Hoffmann,grid.7450.6,30
4,Stefan,Pöhlmann,grid.411984.1,20


## Create table with author full names

Table: __GOE_AUTHORS_FULL_NAMES__

- Create a new table where author initials and last name are combined to a `full_name` to identify authors
- Note: Middle names are not considered
- Replace some special characters in author names

In [9]:
df = load(f"{TABLE_FOLDER}{table_name(GOE_AUTHORS_AFFILIATION)}.csv")

In [10]:
# dictionary with special characters to replace
REPLACE_CHARS = {
    'ä': 'ae', 'ö': 'oe', 'ü': 'ue', '\.': '', 'ß': 'ss', 'ş': 's', 'ğ': 'g', 'ç': 'c'
}

df_full_name = df.copy()

df_full_name["full_name"] = df_full_name["first_name"].str.lower().str[0] + "_" + df_full_name["last_name"].str.lower()
df_full_name.replace({'full_name': REPLACE_CHARS}, regex=True, inplace=True)
df_full_name.head(5)

,first_name,last_name,grid_iden,freq,full_name
0,Stefan,Pöhlmann,grid.418215.b,44,s_poehlmann
1,Stefan,Pöhlmann,grid.7450.6,41,s_poehlmann
2,Markus,Hoffmann,grid.418215.b,40,m_hoffmann
3,Markus,Hoffmann,grid.7450.6,30,m_hoffmann
4,Stefan,Pöhlmann,grid.411984.1,20,s_poehlmann


In [11]:
bqupload(
    client, df_full_name, GOE_AUTHORS_FULL_NAMES
)

In [12]:
save(df_full_name, f"{TABLE_FOLDER}{table_name(GOE_AUTHORS_FULL_NAMES)}.csv")

## Create table with location information for each affiliation for each author

Table: __GOE_AUTHORS_LOCATION__

- An author can have multiple affiliations; all of them are considered in this table

In [13]:
sql = f"""
SELECT
  goe.full_name,
  goe.grid_iden,
  goe.freq,
  ds.name,
  ds.address.city as city,
  ds.address.country as country
FROM {GOE_AUTHORS_FULL_NAMES} AS goe
INNER JOIN {DS_GRID} AS ds
    ON `ds`.`id` = `goe`.`grid_iden`
ORDER BY freq DESC
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_AUTHORS_LOCATION)
)

In [14]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_AUTHORS_LOCATION)}.csv")
df.head(5)

,full_name,grid_iden,freq,name,city,country
0,s_poehlmann,grid.418215.b,44,German Primate Center,Göttingen,Germany
1,s_poehlmann,grid.7450.6,41,University of Göttingen,Göttingen,Germany
2,m_hoffmann,grid.418215.b,40,German Primate Center,Göttingen,Germany
3,m_hoffmann,grid.7450.6,30,University of Göttingen,Göttingen,Germany
4,s_poehlmann,grid.411984.1,20,Universitätsmedizin Göttingen,Göttingen,Germany


## Create table with main affiliation for each author

Table: __GOE_AUTHORS_MAIN_AFFILIATION__

- An author can have multiple affiliations. For later use, only the affiliation for with an author wrote most papers is considered in this table.
- If an author has multiple affiliations with the same number of papers, the first affiliation in the DataFrame is considered.

In [15]:
df = load(f"{TABLE_FOLDER}{table_name(GOE_AUTHORS_LOCATION)}.csv")

In [16]:
df_main_affiliation = df.copy()
df_main_affiliation = df_main_affiliation.groupby(["full_name", "grid_iden"])["freq"].max().reset_index()
df_main_affiliation = df_main_affiliation.sort_values('freq', ascending=False)
df_main_affiliation = df_main_affiliation.drop_duplicates(subset='full_name', keep='first')
df_main_affiliation.rename(columns = {'grid_iden': 'main_affiliation'}, inplace=True)
df_main_affiliation.head(5)

,full_name,main_affiliation,freq
3465,s_poehlmann,grid.418215.b,44
2401,m_hoffmann,grid.418215.b,40
2140,l_gattinoni,grid.7450.6,17
1446,h_kleine-weber,grid.418215.b,15
1436,h_hofmann-winkler,grid.418215.b,14


In [17]:
df_main_affiliation.drop(['freq'], axis=1, inplace=True)
df_main_affiliation.head()

,full_name,main_affiliation
3465,s_poehlmann,grid.418215.b
2401,m_hoffmann,grid.418215.b
2140,l_gattinoni,grid.7450.6
1446,h_kleine-weber,grid.418215.b
1436,h_hofmann-winkler,grid.418215.b


In [18]:
bqupload(
    client, df_main_affiliation, GOE_AUTHORS_MAIN_AFFILIATION
)

In [19]:
save(df_main_affiliation, f"{TABLE_FOLDER}{table_name(GOE_AUTHORS_MAIN_AFFILIATION)}.csv")

-----

# Part 3: Cooperations

## Create table with authors and their main affiliations for each publication

Table: __GOE_AUTHORS_COOPERATION__

- SQL statement selects publication ID, author full names, and the main affiliation with its name, city, and country for each author

In [20]:
sql = f"""
SELECT DISTINCT
    goe.id,
    fn.full_name,
    loc.name as affiliation_name,
    loc.city as affiliation_city,
    loc.country as affiliation_country,
    aff.main_affiliation as affiliation_id
FROM {GOE_AUTHORS} AS goe
LEFT JOIN 
    UNNEST(goe.authors) AS a
JOIN {GOE_AUTHORS_FULL_NAMES} AS fn
    ON (fn.first_name = a.first_name AND fn.last_name = a.last_name)
JOIN {GOE_AUTHORS_LOCATION} AS loc
    ON (fn.full_name = loc.full_name)
JOIN {GOE_AUTHORS_MAIN_AFFILIATION} AS aff
    ON (fn.full_name = aff.full_name)
WHERE
    loc.grid_iden = aff.main_affiliation
ORDER BY id
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_AUTHORS_COOPERATION)
)

In [21]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_AUTHORS_COOPERATION)}.csv")
df.head(5)

,id,full_name,affiliation_name,affiliation_city,affiliation_country,affiliation_id
0,pub.1124079136,b_ahmed,Cairo University,Giza,Egypt,grid.7776.1
1,pub.1124079136,h_amer,Cairo University,Giza,Egypt,grid.7776.1
2,pub.1124079136,j_kissenkoetter,University of Göttingen,Göttingen,Germany,grid.7450.6
3,pub.1124079136,a_el wahed,University of Göttingen,Göttingen,Germany,grid.7450.6
4,pub.1124079136,m_bayoumi,Cairo University,Giza,Egypt,grid.7776.1


## Create table that counts cooperations for institutions from Göttingen campus

Table: __GOE_COOPERATIONS__

- SQL statement selects affiliation information and counts occurrence of these affiliations in the authors cooperation table
- By selecting and grouping different columns (e.g., only `affiliation_country`) the cooperation count can be conducted on an institutional, city or country level.

In [22]:
sql = f"""
SELECT
    affiliation_name,
    affiliation_city,
    affiliation_country,
    count(DISTINCT id) AS nr_of_cooperations
FROM {GOE_AUTHORS_COOPERATION} as goe
WHERE
    affiliation_id NOT IN {GOE_CAMPUS_IDS}
GROUP BY 
    affiliation_name,
    affiliation_city,
    affiliation_country
ORDER BY nr_of_cooperations DESC
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_COOPERATIONS)
)

In [23]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_COOPERATIONS)}.csv")
df.head(5)

,affiliation_name,affiliation_city,affiliation_country,nr_of_cooperations
0,Charité - University Medicine Berlin,Berlin,Germany,37
1,Hannover Medical School,Hanover,Germany,37
2,University of Erlangen-Nuremberg,Erlangen,Germany,23
3,Ludwig-Maximilians-Universität München,Munich,Germany,18
4,Jena University Hospital,Jena,Germany,18


-----

# Part 4: Categories

### Count occurrences of category fields

Table: __GOE_CATEGORY_COUNT__

- SQL statement selects counts for each category field for each publication
- For each column (i.e., category), the sum of publications with information in this category is counted afterwards
- We get an overview of how many publications have a value for a specific category

In [24]:
sql = f"""
SELECT
Sum(case When bra_v1 > 0 Then 1 Else 0 End) AS bra_v1,
Sum(case When for_v1_first_level > 0 Then 1 Else 0 End) AS for_v1_first_level,
Sum(case When for_v1_second_level > 0 Then 1 Else 0 End) AS for_v1_second_level,
Sum(case When hra_v1 > 0 Then 1 Else 0 End) AS hra_v1,
Sum(case When hrcs_hc_v1 > 0 Then 1 Else 0 End) AS hrcs_hc_v1,
Sum(case When hrcs_rac_v1 > 0 Then 1 Else 0 End) AS hrcs_rac_v1,
Sum(case When icrp_cso_v1 > 0 Then 1 Else 0 End) AS icrp_cso_v1,
Sum(case When rcdc_v1 > 0 Then 1 Else 0 End) AS rcdc_v1,
Sum(case When sdg_v1 > 0 Then 1 Else 0 End) AS sdg_v1,
Sum(case When uoa_v1 > 0 Then 1 Else 0 End) AS uoa_v1 
FROM
(
SELECT 
    goe.id,
    count(bra_v1.value) AS bra_v1,
    count(for_v1_first.name) AS for_v1_first_level,
    count(for_v1_second.name) AS for_v1_second_level,
    count(hra_v1.value) AS hra_v1,
    count(hrcs_hc_v1.value) AS hrcs_hc_v1,
    count(hrcs_rac_v1.name) AS hrcs_rac_v1,
    count(icrp_cso_v1.name) AS icrp_cso_v1,
    count(rcdc_v1.value) AS rcdc_v1,
    count(sdg_v1.name) AS sdg_v1,
    count(uoa_v1.name) AS uoa_v1 
FROM {GOE_COVID} AS goe
LEFT JOIN
    UNNEST(goe.categories.bra_v1.full) AS bra_v1
LEFT JOIN 
    UNNEST(goe.categories.for_v1.first_level.full) AS for_v1_first
LEFT JOIN 
    UNNEST(goe.categories.for_v1.second_level.full) AS for_v1_second
LEFT JOIN 
    UNNEST(goe.categories.hra_v1.full) AS hra_v1
LEFT JOIN 
    UNNEST(goe.categories.hrcs_hc_v1.full) AS hrcs_hc_v1
LEFT JOIN 
    UNNEST(goe.categories.hrcs_rac_v1.full) AS hrcs_rac_v1
LEFT JOIN 
    UNNEST(goe.categories.icrp_cso_v1.full) AS icrp_cso_v1
LEFT JOIN 
    UNNEST(goe.categories.rcdc_v1.full) AS rcdc_v1
LEFT JOIN 
    UNNEST(goe.categories.sdg_v1.full) AS sdg_v1
LEFT JOIN 
    UNNEST(goe.categories.uoa_v1.full) AS uoa_v1
GROUP BY
    goe.id
ORDER BY goe.id
)
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_CATEGORY_COUNT)
)

In [25]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_CATEGORY_COUNT)}.csv")
df

,bra_v1,for_v1_first_level,for_v1_second_level,hra_v1,hrcs_hc_v1,hrcs_rac_v1,icrp_cso_v1,rcdc_v1,sdg_v1,uoa_v1
0,215,426,391,206,194,114,14,274,65,360


## Collect all categories for all publications

Table: __GOE_CATEGORIES__

- SQL statement selects category values for each publication ID in their appropriate column
- Note: The same publication can have multiple rows in the resulting table!
- Note: We use only FOR and SDG categories, for further details see https://docs.dimensions.ai/dsl/datasource-publications.html and https://dimensions.freshdesk.com/support/solutions/articles/23000018820-what-are-fields-of-research-and-other-classification-systems-and-how-are-they-created-

In [28]:
sql = f"""
SELECT 
    goe.id,
    # bra_v1.value AS bra_v1,
    for_v1_first.name AS for_v1_first_level,
    for_v1_second.name AS for_v1_second_level,
    # hra_v1.value AS hra_v1,
    # hrcs_hc_v1.value AS hrcs_hc_v1,
    # hrcs_rac_v1.name AS hrcs_rac_v1,
    # icrp_cso_v1.name AS icrp_cso_v1,
    # rcdc_v1.value AS rcdc_v1,
    sdg_v1.name AS sdg_v1,
    # uoa_v1.name AS uoa_v1 
FROM {GOE_COVID} AS goe
# LEFT JOIN
#     UNNEST(goe.categories.bra_v1.full) AS bra_v1
LEFT JOIN 
    UNNEST(goe.categories.for_v1.first_level.full) AS for_v1_first
LEFT JOIN 
    UNNEST(goe.categories.for_v1.second_level.full) AS for_v1_second
# LEFT JOIN 
#     UNNEST(goe.categories.hra_v1.full) AS hra_v1
# LEFT JOIN 
#     UNNEST(goe.categories.hrcs_hc_v1.full) AS hrcs_hc_v1
# LEFT JOIN 
#     UNNEST(goe.categories.hrcs_rac_v1.full) AS hrcs_rac_v1
# LEFT JOIN 
#     UNNEST(goe.categories.icrp_cso_v1.full) AS icrp_cso_v1
# LEFT JOIN 
#     UNNEST(goe.categories.rcdc_v1.full) AS rcdc_v1
LEFT JOIN 
    UNNEST(goe.categories.sdg_v1.full) AS sdg_v1
# LEFT JOIN 
#     UNNEST(goe.categories.uoa_v1.full) AS uoa_v1
GROUP BY 
    goe.id,
    # bra_v1,
    for_v1_first_level,
    for_v1_second_level,
    # hra_v1,
    # hrcs_hc_v1,
    # hrcs_rac_v1,
    # icrp_cso_v1,
    # rcdc_v1,
    sdg_v1
    # uoa_v1 
ORDER BY goe.id
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_CATEGORIES)
)

In [29]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_CATEGORIES)}.csv")
df.head(5)

,id,for_v1_first_level,for_v1_second_level,sdg_v1
0,pub.1124079136,Chemical Sciences,Medicinal and Biomolecular Chemistry,None
1,pub.1124079136,Chemical Sciences,Biochemistry and Cell Biology,None
2,pub.1124079136,Biological Sciences,Medicinal and Biomolecular Chemistry,None
3,pub.1124079136,Biological Sciences,Biochemistry and Cell Biology,None
4,pub.1124214819,Biological Sciences,Microbiology,None


## Create table with category values assigned to each single publication

Table: __GOE_CATEGORIES_PUBLICATIONS__

- SQL statements combines values from all categories to a single column

In [32]:
sql = f"""
SELECT
    DISTINCT *
FROM (
    # (SELECT DISTINCT id, bra_v1 AS category FROM {GOE_CATEGORIES}
    # WHERE bra_v1 IS NOT NULL)
    # UNION ALL
    (SELECT DISTINCT id, for_v1_first_level AS category FROM {GOE_CATEGORIES}
    WHERE for_v1_first_level IS NOT NULL)
    UNION ALL
    (SELECT DISTINCT id, for_v1_second_level AS category FROM {GOE_CATEGORIES}
    WHERE for_v1_second_level IS NOT NULL)
    UNION ALL
    # (SELECT DISTINCT id, hra_v1 AS category FROM {GOE_CATEGORIES}
    # WHERE hra_v1 IS NOT NULL)
    # UNION ALL
    # (SELECT DISTINCT id, hrcs_hc_v1 AS category FROM {GOE_CATEGORIES}
    # WHERE hrcs_hc_v1 IS NOT NULL)
    # UNION ALL
    # (SELECT DISTINCT id, hrcs_rac_v1 AS category FROM {GOE_CATEGORIES}
    # WHERE hrcs_rac_v1 IS NOT NULL)
    # UNION ALL
    # (SELECT DISTINCT id, icrp_cso_v1 AS category FROM {GOE_CATEGORIES}
    # WHERE icrp_cso_v1 IS NOT NULL)
    # UNION ALL
    # (SELECT DISTINCT id, rcdc_v1 AS category FROM {GOE_CATEGORIES}
    # WHERE rcdc_v1 IS NOT NULL)
    # UNION ALL
    (SELECT DISTINCT id, sdg_v1 AS category FROM {GOE_CATEGORIES}
    WHERE sdg_v1 IS NOT NULL)
    # UNION ALL
    # (SELECT DISTINCT id, uoa_v1 AS category FROM {GOE_CATEGORIES}
    # WHERE uoa_v1 IS NOT NULL)
    ORDER BY id
)
ORDER BY id
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_CATEGORIES_PUBLICATIONS)
)

In [33]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_CATEGORIES_PUBLICATIONS)}.csv")
df.head(5)

,id,category
0,pub.1124079136,Medicinal and Biomolecular Chemistry
1,pub.1124079136,Biochemistry and Cell Biology
2,pub.1124079136,Chemical Sciences
3,pub.1124079136,Biological Sciences
4,pub.1124214819,Microbiology


In [34]:
# look into number of papers for each category
Counter(df["category"].values).most_common()[:10]

[('Medical and Health Sciences', 291),
 ('Clinical Sciences', 94),
 ('Public Health and Health Services', 69),
 ('Biological Sciences', 55),
 ('Immunology', 53),
 ('Medical Microbiology', 44),
 ('Good Health and Well Being', 43),
 ('Biochemistry and Cell Biology', 33),
 ('Cardiorespiratory Medicine and Haematology', 22),
 ('Economics', 22)]

## Create table with each category assigned to each affiliation for each paper

Table: __GOE_CATEGORIES_AFFILIATIONS__

- SQL statement selects publication ID, main affiliation of its authors and all category values from this publication for each of its affiliations
- We then can get a table of affiliations and the categories of papers their are involved in

In [35]:
sql = f"""
SELECT DISTINCT
    cat.id,
    aff.main_affiliation,
    cat.category,
FROM {GOE_AUTHORS} as goe
JOIN
    UNNEST(goe.authors) AS a
JOIN {GOE_CATEGORIES_PUBLICATIONS} as cat
    ON goe.id = cat.id
JOIN {GOE_AUTHORS_FULL_NAMES} AS fn
    ON (a.first_name = fn.first_name AND a.last_name = fn.last_name)
JOIN {GOE_AUTHORS_MAIN_AFFILIATION} as aff
    ON (fn.full_name = aff.full_name)
WHERE
    aff.main_affiliation NOT IN  {GOE_CAMPUS_IDS}
ORDER BY cat.id
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_CATEGORIES_AFFILIATIONS)
)

In [36]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_CATEGORIES_AFFILIATIONS)}.csv")
df.head(5)

,id,main_affiliation,category
0,pub.1124079136,grid.7776.1,Medicinal and Biomolecular Chemistry
1,pub.1124079136,grid.7776.1,Biochemistry and Cell Biology
2,pub.1124079136,grid.7776.1,Chemical Sciences
3,pub.1124079136,grid.7776.1,Biological Sciences
4,pub.1124079136,grid.418376.f,Medicinal and Biomolecular Chemistry


## Create table with category count for each category for each affiliation

Table: __GOE_CATEGORIES_AFFILIATIONS_COUNT__

- SQL statement groups affiliations and categories by counting occurrence of a category value for each affiliation

In [37]:
sql = f"""
SELECT
    fosaff.main_affiliation,
    fosaff.category,
    count(fosaff.category) as category_count
FROM {GOE_CATEGORIES_AFFILIATIONS} as fosaff
GROUP BY
    fosaff.main_affiliation,
    fosaff.category
ORDER BY category_count DESC
"""

q = client.query(sql)
client.query(
    bqcreate(sql, GOE_CATEGORIES_AFFILIATIONS_COUNT)
)

In [38]:
df = q.to_dataframe()
save(df, f"{TABLE_FOLDER}{table_name(GOE_CATEGORIES_AFFILIATIONS_COUNT)}.csv")
df.head(5)

,main_affiliation,category,category_count
0,grid.6363.0,Medical and Health Sciences,30
1,grid.10423.34,Medical and Health Sciences,24
2,grid.5330.5,Medical and Health Sciences,18
3,grid.4708.b,Medical and Health Sciences,14
4,grid.17635.36,Medical and Health Sciences,14


In [39]:
# look into number of institutions for each category
Counter(df["category"].values).most_common()[:10]

[('Medical and Health Sciences', 646),
 ('Clinical Sciences', 268),
 ('Public Health and Health Services', 234),
 ('Biological Sciences', 174),
 ('Good Health and Well Being', 164),
 ('Immunology', 146),
 ('Cardiorespiratory Medicine and Haematology', 115),
 ('Medical Microbiology', 98),
 ('Biochemistry and Cell Biology', 97),
 ('Neurosciences', 66)]